# 🧹 TCS Stock Data - Data Cleaning & Exploratory Data Analysis

## Objective
This notebook focuses on:
- Data cleaning and preprocessing
- Missing value handling
- Outlier detection and treatment
- Comprehensive exploratory data analysis
- Interactive visualizations with Plotly

---

## 📦 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from datetime import datetime, timedelta
from scipy import stats
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

print("✅ Libraries imported successfully!")

## 📁 Load and Initial Setup

In [ ]:
# Load the TCS stock data
df = pd.read_csv('../data/TCS_stock_history.csv')
print(f"✅ Data loaded: {df.shape}")

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# Create a copy for cleaning
df_original = df.copy()
print(f"📊 Original data shape: {df_original.shape}")

df.head()

## 🧹 Data Cleaning Process

In [ ]:
print("=" * 50)
print("🧹 DATA CLEANING PROCESS")
print("=" * 50)

# 1. Check for missing values
print("1. Missing Values Analysis:")
missing_before = df.isnull().sum()
print(missing_before)

# 2. Handle missing values if any
if df.isnull().sum().sum() > 0:
    print("\n🔧 Handling missing values...")
    # Forward fill then backward fill
    df = df.fillna(method='ffill').fillna(method='bfill')
    print(f"✅ Missing values after cleaning: {df.isnull().sum().sum()}")
else:
    print("✅ No missing values found")

# 3. Remove duplicates
print("\n2. Duplicate Records:")
duplicates_before = df.duplicated().sum()
print(f"Duplicates before: {duplicates_before}")

if duplicates_before > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print(f"✅ Duplicates after cleaning: {df.duplicated().sum()}")
else:
    print("✅ No duplicates found")

# 4. Data type optimization
print("\n3. Data Type Optimization:")
print("Before:")
print(df.dtypes)

# Optimize numeric columns
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nAfter optimization:")
print(df.dtypes)

print(f"\n📊 Final cleaned data shape: {df.shape}")

## 🔍 Outlier Detection & Treatment

In [ ]:
def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

def detect_outliers_zscore(data, column, threshold=3):
    """Detect outliers using Z-score method"""
    z_scores = np.abs(stats.zscore(data[column]))
    outliers = data[z_scores > threshold]
    return outliers

# Analyze outliers for each numeric column
print("=" * 50)
print("🔍 OUTLIER DETECTION & ANALYSIS")
print("=" * 50)

outlier_summary = {}
price_volume_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

for col in price_volume_cols:
    if col in df.columns:
        # IQR method
        outliers_iqr, lower, upper = detect_outliers_iqr(df, col)
        
        # Z-score method
        outliers_zscore = detect_outliers_zscore(df, col)
        
        outlier_summary[col] = {
            'IQR_outliers': len(outliers_iqr),
            'IQR_percentage': len(outliers_iqr) / len(df) * 100,
            'ZScore_outliers': len(outliers_zscore),
            'ZScore_percentage': len(outliers_zscore) / len(df) * 100,
            'lower_bound': lower,
            'upper_bound': upper
        }
        
        print(f"\n{col}:")
        print(f"  IQR outliers: {len(outliers_iqr)} ({len(outliers_iqr)/len(df)*100:.1f}%)")
        print(f"  Z-Score outliers: {len(outliers_zscore)} ({len(outliers_zscore)/len(df)*100:.1f}%)")
        print(f"  IQR bounds: [{lower:.2f}, {upper:.2f}]")

# Create outlier summary DataFrame
outlier_df = pd.DataFrame(outlier_summary).T
print("\n📊 Outlier Summary:")
display(outlier_df)

## 📊 Comprehensive EDA - Price Analysis

In [ ]:
# Create comprehensive price analysis visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('TCS Stock Price Trend', 'Price Volatility Analysis',
                   'OHLC Candlestick Chart', 'Price Range Analysis'),
    specs=[[{'secondary_y': True}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]]
)

# 1. Price trend with volume
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Close'], name='Close Price',
              line=dict(color='#2E86AB', width=2)),
    row=1, col=1, secondary_y=False
)

fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Volume'], name='Volume',
              line=dict(color='#A23B72', width=1, dash='dot')),
    row=1, col=1, secondary_y=True
)

# 2. Daily price change
df['Daily_Change'] = df['Close'].pct_change() * 100
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Daily_Change'], name='Daily % Change',
              line=dict(color='#F18F01', width=1)),
    row=1, col=2
)

# 3. Candlestick chart (sample last 100 days)
sample_df = df.tail(100)
fig.add_trace(
    go.Candlestick(x=sample_df['Date'],
                   open=sample_df['Open'],
                   high=sample_df['High'],
                   low=sample_df['Low'],
                   close=sample_df['Close'],
                   name='OHLC'),
    row=2, col=1
)

# 4. Price range (High-Low)
df['Price_Range'] = df['High'] - df['Low']
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Price_Range'], name='Daily Range',
              line=dict(color='#C73E1D', width=1)),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="TCS Stock - Comprehensive Price Analysis",
    title_x=0.5,
    showlegend=True
)

# Update y-axes labels
fig.update_yaxes(title_text="Price (₹)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="% Change", row=1, col=2)
fig.update_yaxes(title_text="Price (₹)", row=2, col=1)
fig.update_yaxes(title_text="Range (₹)", row=2, col=2)

fig.show()

## 📈 Volume Analysis

In [ ]:
# Volume analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Volume Over Time', 'Volume Distribution',
                   'Price vs Volume Relationship', 'Volume Anomalies'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]]
)

# 1. Volume over time
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Volume'], name='Volume',
              line=dict(color='#FF6B35', width=1)),
    row=1, col=1
)

# 2. Volume distribution
fig.add_trace(
    go.Histogram(x=df['Volume'], name='Volume Distribution',
                marker_color='#004E89', nbinsx=50),
    row=1, col=2
)

# 3. Price vs Volume scatter
fig.add_trace(
    go.Scatter(x=df['Volume'], y=df['Close'], mode='markers',
              name='Price vs Volume',
              marker=dict(color='#009639', size=4, opacity=0.6)),
    row=2, col=1
)

# 4. Volume anomalies (extremely high/low volume days)
volume_mean = df['Volume'].mean()
volume_std = df['Volume'].std()
high_volume = df[df['Volume'] > (volume_mean + 2 * volume_std)]
low_volume = df[df['Volume'] < (volume_mean - 2 * volume_std)]

fig.add_trace(
    go.Scatter(x=high_volume['Date'], y=high_volume['Volume'],
              mode='markers', name='High Volume Days',
              marker=dict(color='red', size=8)),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(x=low_volume['Date'], y=low_volume['Volume'],
              mode='markers', name='Low Volume Days',
              marker=dict(color='blue', size=8)),
    row=2, col=2
)

# Add average volume line
fig.add_hline(y=volume_mean, line_dash="dash", line_color="gray",
              annotation_text="Average Volume", row=2, col=2)

fig.update_layout(
    height=800,
    title_text="TCS Stock - Volume Analysis",
    title_x=0.5,
    showlegend=True
)

fig.show()

# Volume statistics
print("📊 Volume Statistics:")
print(f"Average Volume: {df['Volume'].mean():,.0f}")
print(f"Median Volume: {df['Volume'].median():,.0f}")
print(f"Max Volume: {df['Volume'].max():,.0f}")
print(f"Min Volume: {df['Volume'].min():,.0f}")
print(f"High Volume Days (>2σ): {len(high_volume)}")
print(f"Low Volume Days (<-2σ): {len(low_volume)}")

## 🔥 Correlation Analysis

In [ ]:
# Correlation analysis
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
correlation_matrix = df[numeric_cols].corr()

# Create interactive correlation heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.round(3).values,
    texttemplate="%{text}",
    textfont={"size": 12},
    hoverongaps=False
))

fig.update_layout(
    title="TCS Stock - Correlation Heatmap",
    title_x=0.5,
    width=600,
    height=500
)

fig.show()

# Display correlation insights
print("🔍 Key Correlation Insights:")
print(f"Open-Close correlation: {correlation_matrix.loc['Open', 'Close']:.3f}")
print(f"High-Low correlation: {correlation_matrix.loc['High', 'Low']:.3f}")
print(f"Volume-Close correlation: {correlation_matrix.loc['Volume', 'Close']:.3f}")
print(f"High-Close correlation: {correlation_matrix.loc['High', 'Close']:.3f}")
print(f"Low-Close correlation: {correlation_matrix.loc['Low', 'Close']:.3f}")

## 📅 Temporal Analysis

In [ ]:
# Add temporal features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Quarter'] = df['Date'].dt.quarter

# Year-wise analysis
yearly_stats = df.groupby('Year').agg({
    'Close': ['mean', 'min', 'max', 'std'],
    'Volume': ['mean', 'sum'],
    'Date': 'count'
}).round(2)

yearly_stats.columns = ['Avg_Close', 'Min_Close', 'Max_Close', 'Close_Std', 'Avg_Volume', 'Total_Volume', 'Trading_Days']

print("📊 Year-wise Performance:")
display(yearly_stats)

# Monthly seasonality
monthly_returns = df.groupby('Month')['Close'].mean()

# Day of week analysis
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_analysis = df.groupby('DayOfWeek').agg({
    'Close': 'mean',
    'Volume': 'mean',
    'Daily_Change': 'mean'
}).round(2)
dow_analysis.index = [day_names[i] for i in dow_analysis.index]

# Create temporal analysis visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Year-wise Average Close Price', 'Monthly Seasonality',
                   'Day of Week Analysis', 'Quarterly Performance'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': True}, {'secondary_y': False}]]
)

# Year-wise performance
fig.add_trace(
    go.Bar(x=yearly_stats.index, y=yearly_stats['Avg_Close'],
           name='Avg Close Price', marker_color='#1f77b4'),
    row=1, col=1
)

# Monthly seasonality
fig.add_trace(
    go.Scatter(x=monthly_returns.index, y=monthly_returns.values,
              mode='lines+markers', name='Monthly Avg Close',
              line=dict(color='#ff7f0e', width=3)),
    row=1, col=2
)

# Day of week analysis
fig.add_trace(
    go.Bar(x=dow_analysis.index, y=dow_analysis['Close'],
           name='Avg Close', marker_color='#2ca02c'),
    row=2, col=1, secondary_y=False
)

fig.add_trace(
    go.Scatter(x=dow_analysis.index, y=dow_analysis['Volume'],
              mode='lines+markers', name='Avg Volume',
              line=dict(color='#d62728', width=2)),
    row=2, col=1, secondary_y=True
)

# Quarterly performance
quarterly_stats = df.groupby('Quarter')['Close'].mean()
fig.add_trace(
    go.Bar(x=['Q1', 'Q2', 'Q3', 'Q4'], y=quarterly_stats.values,
           name='Quarterly Avg', marker_color='#9467bd'),
    row=2, col=2
)

fig.update_layout(
    height=800,
    title_text="TCS Stock - Temporal Analysis",
    title_x=0.5,
    showlegend=True
)

fig.show()

print("\n📅 Day of Week Analysis:")
display(dow_analysis)

## 💰 Dividends & Stock Splits Analysis

In [ ]:
# Analyze dividends and stock splits
dividend_data = df[df['Dividends'] > 0].copy() if 'Dividends' in df.columns else pd.DataFrame()
splits_data = df[df['Stock Splits'] > 0].copy() if 'Stock Splits' in df.columns else pd.DataFrame()

if not dividend_data.empty or not splits_data.empty:
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Dividends Over Time', 'Stock Splits Timeline'),
        specs=[[{'secondary_y': True}], [{'secondary_y': False}]]
    )
    
    # Plot stock price as background
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['Close'], name='Close Price',
                  line=dict(color='lightgray', width=1), opacity=0.5),
        row=1, col=1, secondary_y=False
    )
    
    if not dividend_data.empty:
        # Dividend events
        fig.add_trace(
            go.Scatter(x=dividend_data['Date'], y=dividend_data['Dividends'],
                      mode='markers+lines', name='Dividends',
                      marker=dict(color='green', size=10, symbol='diamond'),
                      line=dict(color='green', width=2)),
            row=1, col=1, secondary_y=True
        )
        
        print(f"📊 Dividend Summary:")
        print(f"Total dividend events: {len(dividend_data)}")
        print(f"Total dividends paid: ₹{dividend_data['Dividends'].sum():.2f}")
        print(f"Average dividend: ₹{dividend_data['Dividends'].mean():.2f}")
        print(f"Highest dividend: ₹{dividend_data['Dividends'].max():.2f}")
    
    if not splits_data.empty:
        # Stock split events
        fig.add_trace(
            go.Scatter(x=splits_data['Date'], y=splits_data['Stock Splits'],
                      mode='markers', name='Stock Splits',
                      marker=dict(color='purple', size=15, symbol='star')),
            row=2, col=1
        )
        
        print(f"\n📊 Stock Split Summary:")
        print(f"Total split events: {len(splits_data)}")
        print(f"Split ratios: {splits_data['Stock Splits'].unique()}")
    
    fig.update_layout(
        height=600,
        title_text="TCS Stock - Corporate Actions Analysis",
        title_x=0.5
    )
    
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Dividend (₹)", row=1, col=1, secondary_y=True)
    fig.update_yaxes(title_text="Split Ratio", row=2, col=1)
    
    fig.show()
else:
    print("⚠️ No dividend or stock split data available in the dataset")

## 📈 Statistical Analysis Summary

In [ ]:
# Comprehensive statistical summary
print("=" * 60)
print("📈 COMPREHENSIVE STATISTICAL ANALYSIS")
print("=" * 60)

# Price statistics
print("💰 PRICE ANALYSIS:")
print(f"Price Range: ₹{df['Close'].min():.2f} - ₹{df['Close'].max():.2f}")
print(f"Total Return: {((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100:.1f}%")
print(f"Annualized Return: {(((df['Close'].iloc[-1] / df['Close'].iloc[0]) ** (365.25 / (df['Date'].iloc[-1] - df['Date'].iloc[0]).days)) - 1) * 100:.1f}%")
print(f"Average Daily Return: {df['Daily_Change'].mean():.3f}%")
print(f"Volatility (Daily): {df['Daily_Change'].std():.3f}%")
print(f"Annualized Volatility: {df['Daily_Change'].std() * np.sqrt(252):.1f}%")

# Volume statistics
print("\n📊 VOLUME ANALYSIS:")
print(f"Average Daily Volume: {df['Volume'].mean():,.0f} shares")
print(f"Total Volume Traded: {df['Volume'].sum():,.0f} shares")
print(f"Volume Volatility: {df['Volume'].std():,.0f}")

# Risk metrics
print("\n⚠️ RISK METRICS:")
negative_returns = df['Daily_Change'][df['Daily_Change'] < 0]
positive_returns = df['Daily_Change'][df['Daily_Change'] > 0]

print(f"Downside Days: {len(negative_returns)} ({len(negative_returns)/len(df)*100:.1f}%)")
print(f"Upside Days: {len(positive_returns)} ({len(positive_returns)/len(df)*100:.1f}%)")
print(f"Maximum Daily Gain: {df['Daily_Change'].max():.2f}%")
print(f"Maximum Daily Loss: {df['Daily_Change'].min():.2f}%")

# Sharpe ratio (assuming risk-free rate of 5%)
risk_free_rate = 5  # 5% annual
daily_rf = risk_free_rate / 252 / 100
excess_returns = df['Daily_Change'] / 100 - daily_rf
sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
print(f"Sharpe Ratio (5% RF): {sharpe_ratio:.3f}")

# Summary insights
print("\n🎯 KEY INSIGHTS:")
print(f"• Data Quality: {100 - (df.isnull().sum().sum() / df.size * 100):.1f}% complete")
print(f"• Trading Period: {(df['Date'].max() - df['Date'].min()).days} days")
print(f"• Price Trend: {'Bullish' if df['Close'].iloc[-1] > df['Close'].iloc[0] else 'Bearish'}")
print(f"• Volatility Level: {'High' if df['Daily_Change'].std() > 3 else 'Moderate' if df['Daily_Change'].std() > 1.5 else 'Low'}")

print("\n✅ DATA CLEANING & EDA COMPLETED SUCCESSFULLY!")
print("🔄 Next: Feature Engineering (03_feature_engineering.ipynb)")

## 💾 Save Cleaned Data

In [ ]:
# Save cleaned data for next notebooks
df_clean = df.copy()

# Remove temporary columns if any
temp_cols = ['Year', 'Month', 'DayOfWeek', 'Quarter', 'Daily_Change', 'Price_Range']
df_clean = df_clean.drop(columns=[col for col in temp_cols if col in df_clean.columns])

# Save to CSV
df_clean.to_csv('../data/TCS_stock_cleaned.csv', index=False)
print(f"✅ Cleaned data saved: {df_clean.shape}")
print("📁 File: ../data/TCS_stock_cleaned.csv")

# Display final structure
print("\n📋 Final Cleaned Dataset:")
print(f"Shape: {df_clean.shape}")
print(f"Columns: {list(df_clean.columns)}")
print(f"Date Range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")
print(f"Missing Values: {df_clean.isnull().sum().sum()}")